# MPI parallelisation

We will need to install two packages:

- The python MPI package:

```
conda install mpi4py
```

- The actual MPI library:

```
conda install openmpi
```

### References: 
https://mpi4py.readthedocs.io/en/stable/tutorial.html

https://education.molssi.org/parallel-programming/03-distributed-examples-mpi4py/index.html


## 1. Basic python script:

Let's create a .py script that writes "Hello World":


```python
if __name__ == "__main__":

    print("Hello World!")
    
```

To execute it in a terminal:

```
wladimir$ python example_mpi.py

Hello World!
```

To execute it in a terminal:

```
wladimir$ python example1.py 

Hello World!

## 2. Getting Started with MPI

Let’s try running this code on multiple processes. This is done using the **mpiexec** command.

Many environments also provide an **mpirun** command, which usually - but not always - works the same way. 

Whenever possible, you should use mpiexec and not mpirun, in order to guarantee more consistent results.

### MPI - mpiexec vs mpirun

- MPI stands for **‘message passing interface’** and is a message passing standard which is designed to work on a variety of parallel computing architectures.


- The MPI standard defines how syntax and semantics of a library of routines. There are a number of implementations of this standard including OpenMPI, MPICH2, and MS MPI.


- The primary difference between mpiexec and mpirun is that mpiexec is defined as part of the MPI standard, while mpirun is not.


- Different implementations of MPI (i.e. OpenMPI, MPICH, MS MPI, etc.) are not guaranteed to implement mpirun, or might implement different options for mpirun.


- Technically, the MPI standard doesn’t actually require that MPI implementations implement mpiexec either, but the standard does at least describe guidelines for how mpiexec should work. Because of this, mpiexec is generally the preferred command.


The general format for launching a code on multiple processes is:

Let's create a **example2.py** script that writes "Hello World" and execute it in a terminal:


```
wladimir$ mpiexec -n 2 python example_mpi.py
Hello World!
Hello World! 
```

When you execute the above command, mpiexec launches 2 different instances of python example1.py simultaneously, which each print “Hello World!”.

Typically, as long as you have at least 2 processors on the machine you are running on, each process will be launched on a different processor; however, certain environment variables and optional arguments to mpiexec can change this behavior. Each process runs the code in example2.py independently of the others.

### MPI communicator:

It might not be obvious yet, but the processes mpiexec launches aren’t completely unaware of one another. The mpiexec adds each of the processes to an MPI communicator, which enables each of the processes to send and receive information to one another via MPI. The MPI communicator that spans all of the processes launched by mpiexec is called **MPI.COMM_WORLD.**

In mpi4py, communicators are class objects, and we can query information about them through their class functions. Edit example2.py so that it reads as follows:


```python
from mpi4py import MPI

if __name__ == "__main__":

    world_comm = MPI.COMM_WORLD
    world_size = world_comm.Get_size()
    my_rank = world_comm.Get_rank()

    print("World Size: " + str(world_size) + "   " + "Rank: " + str(my_rank))
```

In the above code we first import mpi4py. Then, we get the communicator that spans all of the processes, which is called **MPI.COMM_WORLD.**

The communicator’s **Get_size()** function tells us the total number of processes within that communicator. Each of these processes is assigned a unique rank, which is an integer that ranges from **0** to **world_size - 1**.

The rank of a process allows it to be identified whenever processes communicate with one another. For example, in some cases we might want rank 2 to send some information to rank 4, or we might want rank 0 to receive information from all of the other processes.

Calling **world_comm.Get_rank()** returns the rank of the process that called it within world_comm.


Then, we execute it in a terminal:

```
wladimir$ mpiexec -n 2 python example2.py
World Size: 2   Rank: 0
World Size: 2   Rank: 1
```


As we can see, the **world_comm.Get_size()** function returns 2, which is the total number of ranks we told mpiexec to run with (through the -n argument). Each of the processes is assigned a rank in the range of 0 to 1.

The ranks may not necessarily print out their messages in order; whichever rank reaches the print function first will print out its message first. If you run the code again with more processors, the ranks are likely to print their messages in a different order.

## 3. Basic Infrastructure

We will now do some work with the script in **example_mpi3.py**, which does some simple math with NumPy arrays:

```python
from mpi4py import MPI
import numpy as np

if __name__ == "__main__":

    N = 10000000

    # initialize a
    a = np.ones( N )

    # initialize b
    b = np.zeros( N )
    for i in range( N ):
        b[i] = 1.0 + i

    # add the two arrays
    for i in range( N ):
        a[i] = a[i] + b[i]

    # average the result
    sum = 0.0
    for i in range( N ):
        sum += a[i]
    average = sum / N

    print("Average: " + str(average))
```


Then, run it:

```
wladimir$ python example3.py
Average: 5000001.5
```


Let’s learn something about which parts of this code account for most of the run time. **MPI4Py** provides a timer, **MPI.Wtime()**, which returns the current walltime. We can use this function to determine how long each section of the code takes to run.

For example, to determine how much time is spent initializing array **a**, do the following:

```python 
    # initialize a
    start_time = MPI.Wtime()
    a = np.ones(N)
    end_time = MPI.Wtime()
    if my_rank == 0:
        print("Initialize a time: " + str(end_time-start_time))
```

As the above code indicates, we don’t really want every rank to print the timings, since that could look messy in the output. Instead, we have only rank 0 print this information. Of course, this requires that we add a few lines near the top of the code to query the rank of each process:

```python
    # get basic information about the MPI communicator
    world_comm = MPI.COMM_WORLD
    world_size = world_comm.Get_size()
    my_rank = world_comm.Get_rank()
    
```

Also determine and print the timings of each of the other sections of the code: the intialization of array **b**, the addition of the two arrays, and the final averaging of the result. Your code should look something like this:

```python 
from mpi4py import MPI
import numpy as np

if __name__ == "__main__":

    # get basic information about the MPI communicator
    world_comm = MPI.COMM_WORLD
    world_size = world_comm.Get_size()
    my_rank = world_comm.Get_rank()

    N = 10000000

    # initialize a
    start_time = MPI.Wtime()
    a = np.ones( N )
    end_time = MPI.Wtime()
    if my_rank == 0:
        print("Initialize a time: " + str(end_time-start_time))

    # initialize b
    start_time = MPI.Wtime()
    b = np.zeros( N )
    for i in range( N ):
        b[i] = 1.0 + i
    end_time = MPI.Wtime()
    if my_rank == 0:
        print("Initialize b time: " + str(end_time-start_time))

    # add the two arrays
    start_time = MPI.Wtime()
    for i in range( N ):
        a[i] = a[i] + b[i]
    end_time = MPI.Wtime()
    if my_rank == 0:
        print("Add arrays time: " + str(end_time-start_time))

    # average the result
    start_time = MPI.Wtime()
    sum = 0.0
    for i in range( N ):
        sum += a[i]
    average = sum / N
    end_time = MPI.Wtime()
    if my_rank == 0:
        print("Average result time: " + str(end_time-start_time))
        print("Average: " + str(average))
```



## 4. Point-to-Point Communication
You can try running this on multiple ranks now:

```
mpiexec -n 2 python example_mpi4.py
```

We can see that running on multiple ranks doesn’t help with the timings, because each rank is duplicating all of the same work. We want the ranks to cooperate on the problem, **with each rank working on a different part of the calculation.**

In this example, that means that different ranks will work on different parts of the arrays **a** and **b**, and then the results on each rank will be summed across all the ranks.

We need to decide what parts of the arrays each of the ranks will work on; this is more generally known as a rank’s workload. Add the following code just before the initialization of array **a**:

```python 
    # determine the workload of each rank
    workloads = [ N // world_size for i in range(world_size) ]
    for i in range( N % world_size ):
        workloads[i] += 1
    my_start = 0
    for i in range( my_rank ):
        my_start += workloads[i]
    my_end = my_start + workloads[my_rank]
```

In the above code, **my_start** and **my_end** represent the range over which each rank will perform mathematical operations on the arrays.

We’ll start by parallelizing the code that averages the result. Update the range of the for loop in this part of the code to the following:

```python 
    for i in range( my_start, my_end ):
    
```

This will ensure that each rank is only calculating elements **my_start** through **my_end** of the sum. We then need the ranks to communicate their individually calculated sums so that we can calculate the global sum. To do this, replace the line **average = sum / N** with:


```python 
    if my_rank == 0:
        world_sum = sum
        for i in range( 1, world_size ):
      	    sum_np = np.empty( 1 )
            world_comm.Recv( [sum_np, MPI.DOUBLE], source=i, tag=77 )
            world_sum += sum_np[0]
        average = world_sum / N
    else:
        sum_np = np.array( [sum] )
        world_comm.Send( [sum_np, MPI.DOUBLE], dest=0, tag=77 )
```

The **MPI.DOUBLE** parameter tells MPI what type of information is being communicated by the **Send** and **Recv** calls. In this case, we are sending an array of double precision numbers. If you are communicating information of a different datatype, consult the following:

![data_types.png](attachment:data_types.png)